<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=a7438bc0f4e621dbed8f337e355c99504d5b78c3f81fd769f903a76d202eb63a
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-12 14:17:46
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  4.95 L
Current:  1.40 C
-------------------
Today PnL: 31.57 K (0.23%)
Current PnL: -28.34 L (-18.31%)
CY Booked + Current PnL: -14.05 L (-9.07%)
-------------------
Total profit:  1.37 L
Total loss:  -29.71 L
-------------------
Total Booked + Current PnL: 12.93 L (10.12%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.21%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.33 C
Est FTT PnL: 92.97 L (66.44%)
Deployed:  1.28 C
Current:  1.40 C
CAGR/XIRR %: 5.07%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,0.03,-16.78,20.27,0.09,17000.0,-16911.0,83866.0,94.66,42.0,M-SC,2.37,253.0,-0.99,0.62,10.18,OX40N,NTT,DURABLES
39,INDIGOPNTS,1408.00,0.19,-12.31,14.06,0.02,21521.0,-21493.0,153066.0,133.26,53.0,M-SC,15.83,234.0,-1.00,1.13,33.14,OX40N,NTT,PAINTS
50,MASFIN,398.61,0.66,-1.39,23.77,22.05,22965.0,-1365.0,96615.0,-15.09,57.0,H-SC,6.95,164.0,-0.06,0.72,40.44,XR,ATH,FINANCE
82,VOLTAS,1530.00,1.03,6.97,11.89,19.69,24388.0,13368.0,205110.0,-3.43,52.0,X-MC,2.59,78.0,0.55,1.52,14.24,XY25,NTT,AC
43,ITC,452.00,-0.62,-2.17,12.89,10.44,25238.0,-4342.0,195796.0,-41.56,42.0,X-LC,1.84,5.0,-0.17,1.45,3.27,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
48,KPIGREEN,730.66,4.50,-15.91,74.76,46.95,78768.0,-19936.0,105361.0,-22.21,41.0,H-SC,6.81,154.0,-0.25,0.78,31.99,MH,ATH,POWER
10,BANDHANBNK,400.00,3.26,-27.83,166.61,92.42,334953.0,-77519.0,201040.0,41.52,51.0,H-SC,8.71,170.0,-0.23,1.49,16.19,XY24,NTT,BANKS
22,DIXON,18931.72,2.96,-14.12,41.58,21.58,50041.0,-19791.0,120348.0,-56.73,36.0,X-MC,7.71,56.0,-0.40,0.89,8.27,X40N,ATH,IT
46,JSWINFRA,342.00,2.00,-8.24,25.57,15.23,47007.0,-16511.0,183836.0,-21.45,46.0,X-MC,8.87,66.0,-0.35,1.36,21.35,X40N,NTT,REALTY
67,SONACOMS,806.63,1.73,-14.97,64.12,39.54,55151.0,-15147.0,86012.0,-33.56,51.0,M-SC,4.62,220.0,-0.27,0.64,21.29,AR,ATH,AUTO


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COFFEEDAY,80.00,-3.53,-45.49,148.14,35.27,91700.0,-51648.0,61901.0,-57.75,30.0,L-SC,11.10,270.0,-0.56,0.46,51.50,XR,NTT,HOTELS
26,GILLETTE,11206.78,-2.28,-4.37,38.47,32.42,93401.0,-11106.0,242790.0,-21.44,38.0,X-SC,9.18,85.0,-0.12,1.80,10.06,X40,ATH,FMCG
33,HINDUNILVR,2922.00,-2.00,-12.11,29.33,13.67,66268.0,-31123.0,225940.0,-18.61,25.0,X-LC,2.41,14.0,-0.47,1.67,6.66,XY25,NTT,FMCG
57,RAJOOENG,143.10,-1.97,-37.96,125.04,39.61,79513.0,-38910.0,63590.0,-57.95,24.0,H-SC,28.07,135.0,-0.49,0.47,0.00,AR,ATH,MISC
63,SFL,1287.00,-1.39,-42.94,121.92,26.63,182425.0,-112591.0,149627.0,13.32,28.0,M-SC,11.09,239.0,-0.62,1.11,0.00,XY24,NTT,MISC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.15,-6.60,122.60,107.91,169069.0,-9741.0,137903.0,-23.79,39.0,M-SC,12.50,216.0,-0.06,1.02,2.82,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,0.19,-12.31,14.06,0.02,21521.0,-21493.0,153066.0,133.26,53.0,M-SC,15.83,234.0,-1.00,1.13,33.14,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,0.03,-16.78,20.27,0.09,17000.0,-16911.0,83866.0,94.66,42.0,M-SC,2.37,253.0,-0.99,0.62,10.18,OX40N,NTT,DURABLES
47,KANSAINER,340.00,0.10,-25.41,52.13,13.47,104855.0,-68526.0,201141.0,-69.28,31.0,H-SC,4.70,158.0,-0.65,1.49,2.27,XY24,NTT,PAINTS
66,SIS,528.00,0.05,-25.46,62.69,21.26,51882.0,-28272.0,82760.0,1950.78,43.0,H-SC,4.79,166.0,-0.54,0.61,11.82,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,0.46,6.83,61.26,72.27,110889.0,11565.0,181013.0,-8.35,68.0,M-LC,2.02,99.0,0.1,1.34,13.42,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,0.46,6.83,61.26,72.27,110889.0,11565.0,181013.0,-8.35,68.0,M-LC,2.02,99.0,0.10,1.34,13.42,XR,NTT,IT
38,INDIAMART,4810.62,-0.54,-4.31,116.03,106.72,136939.0,-5316.0,118020.0,-52.54,30.0,H-SC,2.32,139.0,-0.04,0.87,18.06,AR,ATH,MISC
85,ZYDUSLIFE,1286.17,0.86,-1.88,38.22,35.63,78596.0,-3934.0,205640.0,-15.85,44.0,H-MC,3.10,119.0,-0.05,1.52,14.48,AR,ATH,PHARMA
37,IEX,219.00,0.70,-2.65,52.67,48.63,102528.0,-5292.0,194662.0,-35.29,53.0,H-SC,14.02,136.0,-0.05,1.44,8.60,XR,NTT,MISC
25,FINCABLES,1641.55,-0.15,-6.60,122.60,107.91,169069.0,-9741.0,137903.0,-23.79,39.0,M-SC,12.50,216.0,-0.06,1.02,2.82,OX40N,ATH,CABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,-0.16,-21.75,89.71,48.45,140754.0,-43607.0,156899.0,-27.39,24.0,X-MC,12.70,64.0,-0.31,1.16,2.16,X40N,ATH,FINANCE
33,HINDUNILVR,2922.00,-2.00,-12.11,29.33,13.67,66268.0,-31123.0,225940.0,-18.61,25.0,X-LC,2.41,14.0,-0.47,1.67,6.66,XY25,NTT,FMCG
53,PAGEIND,51605.08,-0.23,-9.66,39.55,26.07,58502.0,-15820.0,147920.0,-33.51,32.0,X-MC,12.40,55.0,-0.27,1.10,0.78,X40,ATH,APPARELS
23,DMART,5391.45,0.10,-8.88,40.60,28.12,77840.0,-18682.0,191725.0,-21.41,32.0,X-LC,5.22,19.0,-0.24,1.42,12.65,X40N,ATH,FMCG
75,TMPV,600.00,0.59,-17.37,72.07,42.18,164606.0,-48012.0,228398.0,-25.52,32.0,X-LC,6.36,3.0,-0.29,1.69,1.54,XY24,NTT,AUTO


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,CAMS,4762.0,0.15,-80.14,528.40,24.79,264279.0,-201841.0,50015.0,-81.09,45.0,X-SC,0.95,86.0,-0.76,0.37,22.27,X40N,NTT,MISC
43,ITC,452.0,-0.62,-2.17,12.89,10.44,25238.0,-4342.0,195796.0,-41.56,42.0,X-LC,1.84,5.0,-0.17,1.45,3.27,X40,NTT,FMCG
8,AWL,485.0,0.30,-21.46,95.45,53.50,226200.0,-64768.0,236983.0,-61.97,34.0,X-MC,2.09,58.0,-0.29,1.76,5.26,XY24,NTT,FMCG
36,ICICIPRULI,790.0,1.56,7.06,22.33,30.97,42253.0,12481.0,189219.0,-18.17,64.0,X-MC,2.15,75.0,0.30,1.40,20.51,X40,ATH,INSURANCE
3,ACC,3906.0,0.12,-25.15,119.31,64.15,212491.0,-59851.0,178100.0,-56.31,35.0,X-SC,2.26,82.0,-0.28,1.32,0.12,XY24,BTT,CEMENT


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.00,0.12,-25.15,119.31,64.15,212491.0,-59851.0,178100.0,-56.31,35.0,X-SC,2.26,82.0,-0.28,1.32,0.12,XY24,BTT,CEMENT
54,PGHH,17907.65,0.57,-5.64,41.75,33.76,79114.0,-11325.0,189495.0,-33.45,38.0,X-MC,5.34,57.0,-0.14,1.40,0.57,X40,ATH,FMCG
53,PAGEIND,51605.08,-0.23,-9.66,39.55,26.07,58502.0,-15820.0,147920.0,-33.51,32.0,X-MC,12.40,55.0,-0.27,1.10,0.78,X40,ATH,APPARELS
11,BATAINDIA,2096.00,-0.05,-38.11,118.46,35.20,94334.0,-49036.0,79634.0,4.40,34.0,X-SC,15.90,92.0,-0.52,0.59,1.15,X40,NTT,FOOTWEAR
31,HAVELLS,2069.16,0.41,-9.84,46.95,32.48,138832.0,-32290.0,295701.0,-14.34,41.0,X-MC,5.69,67.0,-0.23,2.19,1.31,X40,ATH,ELECTRICAL


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,0.51,-29.50,102.60,42.84,47024.0,-19174.0,45832.0,-53.72,43.0,X-SC,36.19,83.0,-0.41,0.34,3.69,XY24,NTT,MISC
15,CAMS,4762.00,0.15,-80.14,528.40,24.79,264279.0,-201841.0,50015.0,-81.09,45.0,X-SC,0.95,86.0,-0.76,0.37,22.27,X40N,NTT,MISC
59,RELAXO,1176.00,-0.06,-47.36,189.33,52.31,144673.0,-68747.0,76413.0,-43.96,43.0,X-SC,6.64,91.0,-0.48,0.57,2.28,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.05,-38.11,118.46,35.20,94334.0,-49036.0,79634.0,4.40,34.0,X-SC,15.90,92.0,-0.52,0.59,1.15,X40,NTT,FOOTWEAR
22,DIXON,18931.72,2.96,-14.12,41.58,21.58,50041.0,-19791.0,120348.0,-56.73,36.0,X-MC,7.71,56.0,-0.40,0.89,8.27,X40N,ATH,IT


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,0.67,-12.69,36.62,19.28,110607.0,-43915.0,302041.0,-23.95,63.0,X-LC,4.93,1.0,-0.40,2.24,11.66,X40,ATH,IT
41,INFY,2275.00,-0.23,8.53,42.70,54.87,145003.0,26698.0,339586.0,-15.65,61.0,X-LC,3.31,2.0,0.18,2.52,17.52,X40,BTT,IT
75,TMPV,600.00,0.59,-17.37,72.07,42.18,164606.0,-48012.0,228398.0,-25.52,32.0,X-LC,6.36,3.0,-0.29,1.69,1.54,XY24,NTT,AUTO
81,VBL,671.64,0.19,-3.16,40.10,35.67,122649.0,-9985.0,305857.0,-14.67,58.0,X-LC,4.63,4.0,-0.08,2.27,10.26,X40N,ATH,FMCG
43,ITC,452.00,-0.62,-2.17,12.89,10.44,25238.0,-4342.0,195796.0,-41.56,42.0,X-LC,1.84,5.0,-0.17,1.45,3.27,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,1.31,-37.51,112.64,32.89,54349.0,-28956.0,48250.0,-695.74,55.0,L-MC,5.87,259.0,-0.53,0.36,32.88,XR,NTT,BANKS
6,ASIANTILES,137.00,1.02,-15.04,113.83,81.67,90726.0,-14107.0,79703.0,7166.67,50.0,L-SC,13.61,271.0,-0.16,0.59,54.35,XR,NTT,CERAMICS
50,MASFIN,398.61,0.66,-1.39,23.77,22.05,22965.0,-1365.0,96615.0,-15.09,57.0,H-SC,6.95,164.0,-0.06,0.72,40.44,XR,ATH,FINANCE
13,BSOFT,831.70,0.64,-21.92,93.10,50.79,102250.0,-30825.0,109828.0,0.12,67.0,H-SC,5.97,151.0,-0.30,0.81,27.88,XR,ATH,IT
39,INDIGOPNTS,1408.00,0.19,-12.31,14.06,0.02,21521.0,-21493.0,153066.0,133.26,53.0,M-SC,15.83,234.0,-1.00,1.13,33.14,OX40N,NTT,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BSOFT,831.7,0.64,-21.92,93.10,50.79,102250.0,-30825.0,109828.0,0.12,67.0,H-SC,5.97,151.0,-0.30,0.81,27.88,XR,ATH,IT
1,ABB,7934.0,0.77,-3.07,50.18,45.57,127248.0,-8035.0,253584.0,-37.68,60.0,H-MC,4.78,120.0,-0.06,1.88,7.90,AR,NTT,ELECTRICAL
56,RAJESHEXPO,518.0,0.15,-62.77,168.80,0.07,86203.0,-86109.0,51068.0,1771.21,58.0,L-SC,2.43,268.0,-1.00,0.38,27.01,OX40N,NTT,JEWELLERY
84,WIPRO,420.0,0.46,6.83,61.26,72.27,110889.0,11565.0,181013.0,-8.35,68.0,M-LC,2.02,99.0,0.10,1.34,13.42,XR,NTT,IT
0,5PAISA,593.0,0.16,-41.36,92.53,12.90,118557.0,-90368.0,128128.0,88.70,43.0,H-SC,11.84,161.0,-0.76,0.95,6.54,OX40N,NTT,FINANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.81
1,25,45.71
2,50,76.71


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.05
MC    30.35
LC    24.63
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.35
X40      22.35
X40N     10.89
XR       10.14
XY25      9.39
AR        9.36
OX40N     7.75
SR        1.02
MH        0.78
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.63
X-MC    23.55
X-LC    21.06
M-SC    12.08
X-SC     6.91
H-MC     5.03
L-SC     1.43
M-MC     1.41
M-LC     1.34
H-LC     1.18
L-LC     1.05
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.29,-7.87,42.75
IT,12.80,-16.64,79.70
FINANCE,9.63,-19.44,69.91
ELECTRICAL,6.26,-11.10,51.80
MISC,6.07,-58.34,123.21
PAINTS,6.05,-13.10,29.90
INSURANCE,4.61,-1.61,37.42
PHARMA,4.10,-0.84,33.11
AUTO,3.44,-23.95,77.09


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3213090.0,21
AR,1307158.0,10
XR,1294456.0,13
X40,1022248.0,14
X40N,979518.0,9
OX40N,744742.0,10
XY25,374191.0,6
SR,282757.0,2
MH,78768.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3654262.0,25
M-SC,1427868.0,15
X-MC,1304619.0,16
X-SC,988939.0,8
X-LC,920914.0,11
H-MC,399660.0,3
L-SC,268629.0,3
M-LC,110889.0,1
H-LC,73743.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1272531.0      6
           AR         901459.0      5
M-SC       XY24       815172.0      6
H-SC       XR         772782.0      7
X-SC       X40N       506696.0      3
X-MC       X40        452411.0      7
           XY24       398501.0      3
X-LC       X40        382102.0      5
H-SC       OX40N      345965.0      4
M-SC       OX40N      312574.0      5
X-SC       XY24       294508.0      3
H-SC       SR         282757.0      2
X-MC       X40N       272333.0      4
X-LC       XY24       238562.0      2
H-MC       AR         205844.0      2
X-LC       X40N       200489.0      2
H-MC       XY24       193816.0      1
X-SC       X40        187735.0      2
L-SC       XR         182426.0      2
X-MC       XY25       181374.0      2
M-SC       XR         174010.0      2
           AR         126112.0      2
M-LC       XR         110889.0      1
X-LC       XY25        99761.0      2
L-SC       OX40N       86203.0      1
H-SC       MH          78768.0      1
H-LC       AR          73743.0      1
M-MC       XY25        55600.0      1
L-MC       XR          54349.0      1
L-LC       XY25        37456.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 42.0 seconds
